In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
df_void = pd.read_csv('../Processed dataset/df_voids.csv')
df_parts = pd.read_csv('../Processed dataset/df_parts.csv')
df_void_new = pd.read_csv('../Processed dataset/df_new_voids.csv')
df_parts_new = pd.read_csv('../Processed dataset/df_new_parts.csv')


In [3]:
df_void_new.head(10)

,Image name,Position,Region type,ID,Area [pixels],Perimeter [pixels],Eccentricity,Orientation [radians],Solidity,Extent,Major Axis Length [pixels],Minor Axis Length [pixels],Equivalent Diameter [pixels],Part_ID,Num_voids
0,img01.bmp,top_left,void,1,104,40.142,0.752,-1.556,0.912,0.693,14.874,9.809,11.507,0,41
1,img01.bmp,top_left,void,2,106,47.799,0.935,-1.335,0.898,0.589,20.548,7.293,11.617,0,41
2,img01.bmp,top_left,void,3,263,80.355,0.939,-1.307,0.895,0.501,32.887,11.318,18.299,0,41
3,img01.bmp,top_left,void,4,211,64.006,0.884,-1.478,0.887,0.601,25.354,11.839,16.391,0,41
4,img01.bmp,top_left,void,5,159,56.355,0.874,0.932,0.888,0.523,21.442,10.408,14.228,0,41
5,img01.bmp,top_left,void,6,211,73.077,0.951,-1.313,0.847,0.471,31.267,9.708,16.391,0,41
6,img01.bmp,top_left,void,7,339,75.841,0.326,0.692,0.929,0.589,22.505,21.278,20.776,0,41
7,img01.bmp,top_left,void,8,361,78.184,0.502,1.001,0.928,0.602,24.343,21.051,21.439,0,41
8,img01.bmp,top_left,void,9,329,77.770,0.718,0.249,0.901,0.580,26.044,18.126,20.467,0,41
9,img01.bmp,top_left,void,10,311,82.083,0.900,-0.839,0.899,0.461,31.733,13.860,19.899,0,41


In [4]:
df_void_new[df_void_new["Part_ID"]  == 0].head(10)

,Image name,Position,Region type,ID,Area [pixels],Perimeter [pixels],Eccentricity,Orientation [radians],Solidity,Extent,Major Axis Length [pixels],Minor Axis Length [pixels],Equivalent Diameter [pixels],Part_ID,Num_voids
0,img01.bmp,top_left,void,1,104,40.142,0.752,-1.556,0.912,0.693,14.874,9.809,11.507,0,41
1,img01.bmp,top_left,void,2,106,47.799,0.935,-1.335,0.898,0.589,20.548,7.293,11.617,0,41
2,img01.bmp,top_left,void,3,263,80.355,0.939,-1.307,0.895,0.501,32.887,11.318,18.299,0,41
3,img01.bmp,top_left,void,4,211,64.006,0.884,-1.478,0.887,0.601,25.354,11.839,16.391,0,41
4,img01.bmp,top_left,void,5,159,56.355,0.874,0.932,0.888,0.523,21.442,10.408,14.228,0,41
5,img01.bmp,top_left,void,6,211,73.077,0.951,-1.313,0.847,0.471,31.267,9.708,16.391,0,41
6,img01.bmp,top_left,void,7,339,75.841,0.326,0.692,0.929,0.589,22.505,21.278,20.776,0,41
7,img01.bmp,top_left,void,8,361,78.184,0.502,1.001,0.928,0.602,24.343,21.051,21.439,0,41
8,img01.bmp,top_left,void,9,329,77.770,0.718,0.249,0.901,0.580,26.044,18.126,20.467,0,41
9,img01.bmp,top_left,void,10,311,82.083,0.900,-0.839,0.899,0.461,31.733,13.860,19.899,0,41


## Info about Johnson Transformation

$$y = \begin{cases} 
      \frac{{(x + 1)^\lambda - 1}}{{\lambda}}, & \text{for } x \geq 0, \lambda \neq 0 \\
      \log(x + 1), & \text{for } x \geq 0, \lambda = 0 \\
      -\frac{{(-x + 1)^{2 - \lambda} - 1}}{{2 - \lambda}}, & \text{for } x < 0, \lambda \neq 2 \\
      -\log(-x + 1), & \text{for } x < 0, \lambda = 2
   \end{cases}$$

## Original image 

In [7]:
start_index = 4

end_index = len(df_void.columns) - 2

img = 11
print(f" Big Voids: {df_void_new[(df_void_new['Part_ID'] == img)].shape}")
print(f"IMG: {img}")
for col in df_void.columns[start_index:end_index]:
    dataset = df_void_new.loc[df_void_new['Part_ID'] == img, col].values  
# Verify the normality of the data
# Shapiro-Wilk test
    _, p_value_SW = stats.shapiro(dataset)
    print('p-value of the Shapiro-Wilk test for "{}": {:.3f}'.format(col, float(p_value_SW)))

    if p_value_SW < 0.05:
        # [data_norm, lmbda] = stats.boxcox(small_void_dataset[col])
        transformed_data, _ = stats.yeojohnson(small_void_dataset[col])
        _, p_value_SW = stats.shapiro(transformed_data)
        print('p-value of the Shapiro-Wilk test for transformed "{}": {:.3f}'.format(col, float(p_value_SW)))

 Big Voids: (46, 15)
IMG: 11
p-value of the Shapiro-Wilk test for "Area [pixels]": 0.070
p-value of the Shapiro-Wilk test for "Perimeter [pixels]": 0.003
p-value of the Shapiro-Wilk test for transformed "Perimeter [pixels]": 0.089
p-value of the Shapiro-Wilk test for "Eccentricity": 0.000
p-value of the Shapiro-Wilk test for transformed "Eccentricity": 0.012
p-value of the Shapiro-Wilk test for "Orientation [radians]": 0.001
p-value of the Shapiro-Wilk test for transformed "Orientation [radians]": 0.028
p-value of the Shapiro-Wilk test for "Solidity": 0.000
p-value of the Shapiro-Wilk test for transformed "Solidity": 0.036
p-value of the Shapiro-Wilk test for "Extent": 0.000
p-value of the Shapiro-Wilk test for transformed "Extent": 0.288
p-value of the Shapiro-Wilk test for "Major Axis Length [pixels]": 0.017
p-value of the Shapiro-Wilk test for transformed "Major Axis Length [pixels]": 0.322
p-value of the Shapiro-Wilk test for "Minor Axis Length [pixels]": 0.028
p-value of the Shapi

## Big voids normality check for each image

In [5]:

img = 11
big_void_dataset = df_void_new[(df_void_new['Part_ID'] == img) & (df_void_new['Area [pixels]'] >= 300)]
print(f"Big Voids: {big_void_dataset.shape}")



print(f"IMG: {img}")
for col in big_void_dataset.columns[start_index:end_index]:
    dataset = big_void_dataset.loc[big_void_dataset['Part_ID'] == img, col].values  
# Verify the normality of the data
# Shapiro-Wilk test
    _, p_value_SW = stats.shapiro(dataset)
    print('p-value of the Shapiro-Wilk test for "{}": {:.3f}'.format(col, float(p_value_SW)))

    if p_value_SW < 0.05:
        # [data_norm, lmbda] = stats.boxcox(small_void_dataset[col])
        transformed_data, _ = stats.yeojohnson(big_void_dataset[col])
        _, p_value_SW = stats.shapiro(transformed_data)
        print('p-value of the Shapiro-Wilk test for transformed "{}": {:.3f}'.format(col, float(p_value_SW)))




Big Voids: (23, 15)
IMG: 11
p-value of the Shapiro-Wilk test for "Area [pixels]": 0.016
p-value of the Shapiro-Wilk test for transformed "Area [pixels]": 0.803
p-value of the Shapiro-Wilk test for "Perimeter [pixels]": 0.008
p-value of the Shapiro-Wilk test for transformed "Perimeter [pixels]": 0.163
p-value of the Shapiro-Wilk test for "Eccentricity": 0.056
p-value of the Shapiro-Wilk test for "Orientation [radians]": 0.031
p-value of the Shapiro-Wilk test for transformed "Orientation [radians]": 0.056
p-value of the Shapiro-Wilk test for "Solidity": 0.798
p-value of the Shapiro-Wilk test for "Extent": 0.375
p-value of the Shapiro-Wilk test for "Major Axis Length [pixels]": 0.147
p-value of the Shapiro-Wilk test for "Minor Axis Length [pixels]": 0.524
p-value of the Shapiro-Wilk test for "Equivalent Diameter [pixels]": 0.057


## Small voids normality check for each image

In [6]:

img = 11
small_void_dataset = df_void_new[(df_void_new['Part_ID'] == img) & (df_void_new['Area [pixels]'] < 300)]
print(f"Small Voids: {small_void_dataset.shape}")

print(f"IMG: {img}")
for col in small_void_dataset.columns[start_index:end_index]:
    dataset = small_void_dataset.loc[small_void_dataset['Part_ID'] == img, col].values  
# Verify the normality of the data
# Shapiro-Wilk test
    _, p_value_SW = stats.shapiro(dataset)
    print('p-value of the Shapiro-Wilk test for "{}": {:.3f}'.format(col, float(p_value_SW)))
    
    if p_value_SW < 0.05:
        # [data_norm, lmbda] = stats.boxcox(small_void_dataset[col])
        transformed_data, _ = stats.yeojohnson(small_void_dataset[col])
        _, p_value_SW = stats.shapiro(transformed_data)
        print('p-value of the Shapiro-Wilk test for transformed "{}": {:.3f}'.format(col, float(p_value_SW)))



Small Voids: (23, 15)
IMG: 11
p-value of the Shapiro-Wilk test for "Area [pixels]": 0.053
p-value of the Shapiro-Wilk test for "Perimeter [pixels]": 0.236
p-value of the Shapiro-Wilk test for "Eccentricity": 0.000
p-value of the Shapiro-Wilk test for transformed "Eccentricity": 0.012
p-value of the Shapiro-Wilk test for "Orientation [radians]": 0.013
p-value of the Shapiro-Wilk test for transformed "Orientation [radians]": 0.028
p-value of the Shapiro-Wilk test for "Solidity": 0.010
p-value of the Shapiro-Wilk test for transformed "Solidity": 0.036
p-value of the Shapiro-Wilk test for "Extent": 0.003
p-value of the Shapiro-Wilk test for transformed "Extent": 0.288
p-value of the Shapiro-Wilk test for "Major Axis Length [pixels]": 0.402
p-value of the Shapiro-Wilk test for "Minor Axis Length [pixels]": 0.501
p-value of the Shapiro-Wilk test for "Equivalent Diameter [pixels]": 0.073
